In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("final_sample.csv")
df

,Unnamed: 0,reviewText,overall
0,76570,"After trying, unsuccessfully, to make iced tea...",1.0
1,129767,This is an awesome seasoning. I bought it on a...,1.0
2,12599,This is a true black tea and brewed with long ...,1.0
3,111185,I went back and bought a second one! Love this...,1.0
4,126497,I am a big tea drinker. I drink caffeinated t...,1.0
...,...,...,...
9995,53718,This item is showing up in a search for Gluten...,0.0
9996,91167,"Okay, so it doesn't taste exactly like pumpkin...",1.0
9997,111963,This mushroom kit is amazing. Although you mu...,1.0
9998,83418,It's decent stuff. We haven't found a lot of ...,1.0


In [3]:
df.drop("Unnamed: 0",   axis='columns', inplace=True)
df

,reviewText,overall
0,"After trying, unsuccessfully, to make iced tea...",1.0
1,This is an awesome seasoning. I bought it on a...,1.0
2,This is a true black tea and brewed with long ...,1.0
3,I went back and bought a second one! Love this...,1.0
4,I am a big tea drinker. I drink caffeinated t...,1.0
...,...,...
9995,This item is showing up in a search for Gluten...,0.0
9996,"Okay, so it doesn't taste exactly like pumpkin...",1.0
9997,This mushroom kit is amazing. Although you mu...,1.0
9998,It's decent stuff. We haven't found a lot of ...,1.0


In [4]:
## Drop Nan values:
df = df.dropna()

In [5]:
#Renaming columns:
df.columns = ["reviewText", "label"]
df.head()

,reviewText,label
0,"After trying, unsuccessfully, to make iced tea...",1.0
1,This is an awesome seasoning. I bought it on a...,1.0
2,This is a true black tea and brewed with long ...,1.0
3,I went back and bought a second one! Love this...,1.0
4,I am a big tea drinker. I drink caffeinated t...,1.0


In [6]:
##Get the independent features
X = df.drop("label", axis=1)

In [7]:
## Get the Dependent features
y=df["label"]

In [8]:
y.value_counts()

1.0    7965
0.0    2034
Name: label, dtype: int64

In [9]:
X.shape

(9999, 1)

In [10]:
y.shape

(9999,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.1.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### Vocabulary size
voc_size=10000

## Data Preprocessing

In [15]:
#Creating a copy of X into the variable messages
messages = X.copy()

In [16]:
messages['reviewText'][0]

'After trying, unsuccessfully, to make iced tea at home I figured I would give it one more try with Lipton.Wow, I made it the way my mom did it years ago, boil a few cups of water steep the tea add some sugar, add more cold water, refrigerate, and when it was ready to drink add the ice and enjoy.I cannot have lemon so I add a nice sprig of mint and it is heaven.  Finally a great iced tea that I will enjoy all year round.'

In [17]:
#Reset index as we drop Nan values from df
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
#In order to remove words that are not meanningful (e.g. the, a, then, often...), we need to download those words.
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Babette\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
##Data Preprocessing/Cleaning
from nltk.stem.porter import PorterStemmer
#Initialise PorterStemmer for Stemming
ps = PorterStemmer()
#Create an empty list named corpus that will contain our cleaned sentences and words
def preprocessing(data):
    corpus = []
#Create a loop to clean all the text in messages:
    for i in range(0, len(messages)):
    #print index
        print(i)
        #use re (regular expressions) to substitute all characters except [a-zA-Z] by blank in message 'reviewText'
        review = re.sub('[^a-zA-Z]', ' ', data[i])
        #use re to substitute multiple spaces, tabs etc by one space
        review = re.sub('/\s\s+/g', ' ', data[i])
        #convert all the characters as lower case
        review = review.lower()
        #split all the words in each sentence to be able to later remove the stopwords
        review = review.split()
    
        #create a loop in review: for each word in review, keep only words that are not stopwords list and apply 'Stemming'
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        #join words with a space to build the review
        review = ' '.join(review)
        #append the review into the corpus
        corpus.append(review)
    return corpus

In [50]:
text = messages['reviewText']
preprocessing(text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339


6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980


8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625


['trying, unsuccessfully, make ice tea home figur would give one tri lipton.wow, made way mom year ago, boil cup water steep tea add sugar, add cold water, refrigerate, readi drink add ice enjoy.i cannot lemon add nice sprig mint heaven. final great ice tea enjoy year round.',
 "awesom seasoning. bought whim . . . . that' bored! shop amazon. bomb. never want without seasoning!",
 'true black tea brew long leaves, way tea be. pyramid bag allow leav expand. diffus ball spoon go troubl bag work better? tea excellent! (along darjeeling) two favorit teas. make late afternoon enjoy plu know fall asleep earli hard day work.',
 'went back bought second one! love this. first bought one made keurig compani hassl fill use. easy. remov basket holder thing keurig, fill littl refil cup put like would k-cup. want &#34;green&#34;, like save money k-cups, much hassle. us actual bought keurig convenience!',
 "big tea drinker. drink caffein tea day, switch white green tea evening. usual drink loos leaf t

In [21]:
#Checking the new sentences in the corpus
for i in range(0,5):
    print(corpus[i])

trying, unsuccessfully, make ice tea home figur would give one tri lipton.wow, made way mom year ago, boil cup water steep tea add sugar, add cold water, refrigerate, readi drink add ice enjoy.i cannot lemon add nice sprig mint heaven. final great ice tea enjoy year round.
awesom seasoning. bought whim . . . . that' bored! shop amazon. bomb. never want without seasoning!
true black tea brew long leaves, way tea be. pyramid bag allow leav expand. diffus ball spoon go troubl bag work better? tea excellent! (along darjeeling) two favorit teas. make late afternoon enjoy plu know fall asleep earli hard day work.
went back bought second one! love this. first bought one made keurig compani hassl fill use. easy. remov basket holder thing keurig, fill littl refil cup put like would k-cup. want &#34;green&#34;, like save money k-cups, much hassle. us actual bought keurig convenience!
big tea drinker. drink caffein tea day, switch white green tea evening. usual drink loos leaf tea, thought i'd gi

## One-Hot representation of words/sentences

In [22]:
#Apply One_hot representation for each word in the corpus based on the voc_size - each word is allocated a number within the sentence.
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[9486,
  1469,
  8581,
  4028,
  1082,
  4573,
  295,
  7360,
  3365,
  3202,
  9458,
  8790,
  4087,
  4490,
  5963,
  6117,
  2733,
  3396,
  6673,
  6357,
  9736,
  8321,
  1082,
  8434,
  9548,
  8434,
  436,
  9736,
  4846,
  9273,
  5806,
  8434,
  4028,
  4679,
  8964,
  3327,
  2525,
  8434,
  6481,
  4133,
  3184,
  2972,
  5540,
  8545,
  4028,
  1082,
  4679,
  2733,
  7329],
 [7129,
  4033,
  3241,
  5620,
  3023,
  1073,
  1175,
  8714,
  8182,
  9623,
  1423,
  9629,
  4033],
 [6940,
  9979,
  1082,
  8568,
  6753,
  3197,
  5963,
  1082,
  8849,
  1538,
  5826,
  9122,
  6366,
  5471,
  9420,
  1823,
  2444,
  2311,
  4463,
  5826,
  4578,
  5485,
  1082,
  5154,
  5715,
  4399,
  4255,
  3069,
  9427,
  8581,
  3387,
  380,
  4679,
  2844,
  2580,
  3278,
  2806,
  3791,
  3464,
  9183,
  4578],
 [3686,
  2502,
  3241,
  1082,
  3202,
  9314,
  1765,
  4497,
  3241,
  3202,
  4490,
  1319,
  1318,
  249,
  4841,
  6397,
  8028,
  9721,
  8345,
  1051,
  4866,
  1319,
 

In [23]:
#Checking the maximum length of all sentences.
number_words=[]
for i in range(0,9999):
    number_words.append(len(onehot_repr[i]))
number_words

[49,
 13,
 41,
 47,
 74,
 11,
 9,
 96,
 103,
 62,
 33,
 19,
 19,
 62,
 24,
 8,
 58,
 11,
 44,
 37,
 22,
 112,
 24,
 89,
 33,
 11,
 36,
 67,
 80,
 25,
 14,
 86,
 14,
 22,
 68,
 22,
 169,
 15,
 28,
 39,
 730,
 65,
 41,
 14,
 28,
 70,
 10,
 66,
 31,
 24,
 75,
 30,
 46,
 14,
 3,
 32,
 61,
 10,
 15,
 97,
 28,
 74,
 97,
 15,
 20,
 53,
 21,
 116,
 82,
 40,
 16,
 61,
 160,
 44,
 14,
 90,
 12,
 162,
 89,
 27,
 23,
 24,
 16,
 93,
 93,
 311,
 49,
 21,
 32,
 59,
 37,
 214,
 52,
 21,
 33,
 63,
 19,
 137,
 13,
 9,
 13,
 24,
 15,
 49,
 26,
 22,
 21,
 91,
 20,
 48,
 16,
 38,
 16,
 34,
 40,
 15,
 39,
 31,
 25,
 16,
 21,
 35,
 22,
 16,
 9,
 9,
 79,
 150,
 111,
 10,
 70,
 12,
 118,
 63,
 24,
 14,
 34,
 19,
 15,
 81,
 14,
 42,
 38,
 12,
 11,
 231,
 183,
 11,
 67,
 54,
 21,
 22,
 192,
 37,
 27,
 46,
 224,
 91,
 151,
 35,
 28,
 28,
 14,
 76,
 12,
 45,
 17,
 68,
 85,
 75,
 32,
 17,
 171,
 97,
 17,
 19,
 18,
 32,
 54,
 26,
 19,
 56,
 88,
 36,
 12,
 51,
 72,
 12,
 17,
 15,
 11,
 12,
 17,
 119,
 34,
 39,
 283,


In [24]:
import numpy as np

In [25]:
max_value = np.max(number_words)
max_value

1134

In [26]:
mean = np.mean(number_words)
mean

51.72847284728473

## Word Embedding

In [27]:
#Use pad sequencing to ensure all sentences are the same length - same number of words
#Set up the common length of each sentence. We decided 100 words even if some sentences are longer
sent_length=100
#Embebbed each sentence as a matrix
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4679 2733 7329]
 [   0    0    0 ... 1423 9629 4033]
 [   0    0    0 ... 3464 9183 4578]
 ...
 [9096 3884 2818 ... 7536  644 4729]
 [   0    0    0 ... 8095 5444 6065]
 [5806 5912 6211 ...  783 5358 5806]]


In [28]:
#Check first sentence embedded with 100 words
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 9486, 1469, 8581, 4028,
       1082, 4573,  295, 7360, 3365, 3202, 9458, 8790, 4087, 4490, 5963,
       6117, 2733, 3396, 6673, 6357, 9736, 8321, 1082, 8434, 9548, 8434,
        436, 9736, 4846, 9273, 5806, 8434, 4028, 4679, 8964, 3327, 2525,
       8434, 6481, 4133, 3184, 2972, 5540, 8545, 4028, 1082, 4679, 2733,
       7329])

## Creating the model Sequential

In [29]:
#Checking the shape of embedded_docs
len(embedded_docs),y.shape

(9999, (9999,))

In [30]:
# #Convert y shape to_categorical
# y = to_categorical(y)
# y.shape

In [31]:
# y.shape[1]

In [32]:
#Storing embedded_docs into an array
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [33]:
X_final.shape, y_final.shape

((9999, 100), (9999,))

In [35]:
##Creating model
#Determining the number of features we want for each vector
embedding_vector_features=40
#Initialise model Sequential
model=Sequential()
#Adding embedding layer: vocabulary size, number of vector features, number of words per sentence
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
#Adding the LSTM layer (Long Short Term Memory) of 100 neurons
model.add(LSTM(100))
#Adding the Dense layer with sigmoid activation
model.add(Dense(1,activation='sigmoid'))
#Compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 40)           400000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [37]:
#Training the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 6699 samples, validate on 3300 samples
Epoch 1/10
6699/6699 [==============================] - 19s 3ms/sample - loss: 0.4852 - accuracy: 0.7947 - val_loss: 0.3824 - val_accuracy: 0.8245
Epoch 2/10
6699/6699 [==============================] - 15s 2ms/sample - loss: 0.3428 - accuracy: 0.8525 - val_loss: 0.3704 - val_accuracy: 0.8409
Epoch 3/10
6699/6699 [==============================] - 15s 2ms/sample - loss: 0.2407 - accuracy: 0.9051 - val_loss: 0.4097 - val_accuracy: 0.8279
Epoch 4/10
6699/6699 [==============================] - 15s 2ms/sample - loss: 0.1777 - accuracy: 0.9345 - val_loss: 0.4709 - val_accuracy: 0.8315
Epoch 5/10
6699/6699 [==============================] - 16s 2ms/sample - loss: 0.1363 - accuracy: 0.9515 - val_loss: 0.5621 - val_accuracy: 0.8242
Epoch 6/10
6699/6699 [==============================] - 16s 2ms/sample - loss: 0.0961 - accuracy: 0.9698 - val_loss: 0.7248 - val_accuracy: 0.8039
Epoch 7/10
6699/6699 [==============================] - 16s 2ms/sample

## Adding Drop out

In [38]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Performance Metrics And Accuracy

In [39]:
y_pred=model.predict_classes(X_test)

In [40]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3300/3300 - 3s - loss: 0.9616 - accuracy: 0.7836
Loss: 0.9615937092809966, Accuracy: 0.7836363911628723


In [41]:
from sklearn.metrics import confusion_matrix

In [42]:
confusion_matrix(y_test,y_pred)

array([[ 297,  353],
       [ 361, 2289]], dtype=int64)

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7836363636363637

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.45      0.46      0.45       650
         1.0       0.87      0.86      0.87      2650

    accuracy                           0.78      3300
   macro avg       0.66      0.66      0.66      3300
weighted avg       0.78      0.78      0.78      3300



## Save the Trained Model

In [45]:
# Save the model
model.save("NLP_sentiment_analysis.h5")

## Evaluate the Model

In [46]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("NLP_sentiment_analysis.h5")

In [48]:
#Testing ML modle on tweet dataset
test = pd.read_csv('tweet_sample.csv')
test

,Unnamed: 0,index,sentiment,id,date,query,user_id,text
0,0,541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,1,750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
2,2,766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
3,3,285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
4,4,705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem
...,...,...,...,...,...,...,...,...
4995,4995,1158200,1,1979246134,Sun May 31 02:13:48 PDT 2009,NO_QUERY,mimikinns,on my way to freedom - feels like i've been tr...
4996,4996,966771,1,1827723007,Sun May 17 11:18:39 PDT 2009,NO_QUERY,Joyhmb3,Minority report I have seen this one but it's ...
4997,4997,47209,0,1677584303,Sat May 02 02:52:50 PDT 2009,NO_QUERY,Trishanson,Oh so tired.. I can't for my pillow and blanke...
4998,4998,1123661,1,1974531134,Sat May 30 13:19:23 PDT 2009,NO_QUERY,DiaZerva,"@satinephoenix not yet, on set but I think it ..."


## Preprocessing test dataset

In [51]:
#Get the independent features
X = test['text']
y= test["sentiment"]

In [52]:
y.value_counts()

1    2504
0    2496
Name: sentiment, dtype: int64

In [53]:
X.shape

(5000,)

In [54]:
y.shape

(5000,)

In [57]:
#Creating a copy of X into the variable messages
messages = X.copy()

In [58]:
#Preprocessing X
preprocessing(X)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086


3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727


['@chrishasboob ahhh hope ok!!!',
 '@misstoriblack cool , tweet app razr 2',
 '@tiannachao know famili drama. lame.hey next time u hang kim n u guy like sleepov whatever, ill call u',
 "school email open geographi stuff revise! *stupid school* :'(",
 'upper airway problem',
 "go miss pastor' sermon faith...",
 'lunch....dj come eat',
 '@piginthepok oh feel like that?',
 'gahh noo!peyton need live!thi horribl',
 '@mrstessyman thank glad like it! product review bit site enjoy knit it!',
 "@perezhilton zach make pee sit down! i'm grown gay man!",
 'sum day one word ......... kackered!',
 '@k9wkj great mind think alik',
 'poorli bed!',
 "@lilpecan oh, that' realli great small blizzard also cold wind blows...",
 '@wize lol, calm down. got 30day loan offer 1500%',
 "i'm feel quit sleepi today, wish could stay bed today...but ok! last year, let' go school",
 '@nadalnew yeah mathieu total choke 3rd set let rog win well. djokov play terribly.',
 "ugh, morning' rough start",
 'bit tongu',
 '@fra

In [60]:
#Checking the new sentences in the corpus
for i in range(0,5):
    print(X[i])

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 


In [61]:
#Apply One_hot representation for each word in the corpus based on the voc_size - each word is allocated a number within the sentence.
onehot_repr=[one_hot(words,voc_size)for words in X] 
onehot_repr

[[2503, 942, 8964, 7695, 3965, 2809],
 [6937, 1141, 8964, 5301, 7841, 3067, 775, 2558, 992, 9486, 3564],
 [4606,
  8964,
  2580,
  9823,
  8580,
  846,
  3715,
  2349,
  8508,
  7661,
  6393,
  8759,
  6958,
  7893,
  1069,
  5712,
  7315,
  8759,
  1073,
  9656,
  5301,
  657,
  2503,
  7127,
  3242,
  6787,
  5764,
  8759],
 [4682,
  7866,
  7438,
  5262,
  7582,
  8964,
  5301,
  7118,
  8385,
  4111,
  6853,
  7215,
  8982,
  4516,
  4682,
  3796],
 [8221, 4573, 1790],
 [9312, 7215, 2912, 8083, 5575, 4111, 1102],
 [4111, 5034, 7440, 9389, 2313, 795, 1069, 1434],
 [7113, 1193, 3015, 4134, 598, 1633, 9656, 7734],
 [2152, 5345, 718, 8091, 7215, 4407, 1765, 3096, 4030],
 [595,
  1248,
  598,
  4071,
  598,
  9656,
  6005,
  6853,
  3096,
  657,
  7781,
  9313,
  125,
  4111,
  7542,
  881,
  4679,
  6055,
  6005],
 [112, 2522, 4464, 1434, 6099, 2121, 1518, 7582, 966, 657, 234, 6128, 5494],
 [7215, 1372, 1468, 992, 9183, 7480, 3202, 936, 2293],
 [286, 8545, 9310, 4399, 2608],
 [3096, 54

In [62]:
#Use pad sequencing to ensure all sentences are the same length - same number of words
#Set up the common length of each sentence. We decided 100 words even if some sentences are longer
sent_length=100
#Embebbed each sentence as a matrix
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 7695 3965 2809]
 [   0    0    0 ...  992 9486 3564]
 [   0    0    0 ... 6787 5764 8759]
 ...
 [   0    0    0 ... 7913 4856 2498]
 [   0    0    0 ... 6416 4791 7875]
 [   0    0    0 ... 9494 7913 9199]]


In [63]:
#Checking the shape of embedded_docs
len(embedded_docs),y.shape

(5000, (5000,))

In [64]:
#Storing embedded_docs into an array
X_tweet_test = np.array(embedded_docs)
y_tweet_test = np.array(y)

In [65]:
X_tweet_test.shape, y_tweet_test.shape

((5000, 100), (5000,))

## Predicting sentiment analysis on Tweet test

In [66]:
y_pred=model.predict_classes(X_tweet_test)

In [67]:
from sklearn.metrics import confusion_matrix

In [69]:
confusion_matrix(y_tweet_test,y_pred)

array([[ 683, 1813],
       [ 552, 1952]], dtype=int64)

In [70]:
from sklearn.metrics import classification_report

In [72]:
from sklearn.metrics import classification_report
print(classification_report(y_tweet_test,y_pred))

              precision    recall  f1-score   support

           0       0.55      0.27      0.37      2496
           1       0.52      0.78      0.62      2504

    accuracy                           0.53      5000
   macro avg       0.54      0.53      0.49      5000
weighted avg       0.54      0.53      0.49      5000



In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(y_tweet_test,y_pred)

0.527